In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_path = "E:\\Datasets\\Pneumonia\\chest_xray\\chest_xray\\train"
val_path = "E:\\Datasets\\Pneumonia\\chest_xray\\chest_xray\\val"
test_path = "E:\\Datasets\\Pneumonia\\chest_xray\\chest_xray\\test"

In [4]:
train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

Found 5216 images belonging to 2 classes.


In [6]:
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(150, 150),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

Found 16 images belonging to 2 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=False  
)

Found 624 images belonging to 2 classes.


In [9]:
from keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adadelta
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, InputLayer
from sklearn.metrics import confusion_matrix, classification_report

In [10]:
def build_vgg16(input_shape=(150, 150, 1), num_classes=2):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(150, activation='relu'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [11]:
vgg16_model = build_vgg16(input_shape=(150, 150, 1), num_classes=2)

C:\Users\Rishi Chaitanya\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
from tensorflow.keras.optimizers import Adam

In [17]:
vgg16_model.compile(
    optimizer=Adam(learning_rate=1e-04),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [19]:
vgg16_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 150, 150, 64)        │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 150, 150, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 75, 75, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 75, 75, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 75, 75, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 37, 37, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 37, 37, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 18, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 18, 18, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 9, 9, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 150)                 │       1,228,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           7,550 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 15,950,138 (60.84 MB)

 Trainable params: 15,950,138 (60.84 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

In [23]:
batch_size = 32
epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [25]:
history = vgg16_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    callbacks=[early_stopping]
)

C:\Users\Rishi Chaitanya\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1006s 6s/step - accuracy: 0.7288 - loss: 0.5936 - val_accuracy: 0.7500 - val_loss: 1.0959
Epoch 2/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/50


C:\Users\Rishi Chaitanya\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
C:\Users\Rishi Chaitanya\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


163/163 ━━━━━━━━━━━━━━━━━━━━ 915s 6s/step - accuracy: 0.8962 - loss: 0.2551 - val_accuracy: 0.6875 - val_loss: 0.8097
Epoch 4/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 990s 6s/step - accuracy: 0.9423 - loss: 0.1636 - val_accuracy: 0.8750 - val_loss: 0.4419
Epoch 6/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1076s 7s/step - accuracy: 0.9501 - loss: 0.1327 - val_accuracy: 0.6250 - val_loss: 0.7080
Epoch 8/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1107s 7s/step - accuracy: 0.9623 - loss: 0.0940 - val_accuracy: 0.8750 - val_loss: 0.3380
Epoch 10/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 660us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/50
163/163 ━━━━━━━━━━━━━━━━━━━━ 1096s 7s/step - accuracy: 0.9731 - loss: 0.0717 - val_accuracy: 0.9

In [32]:
test_loss, test_acc = vgg16_model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.4f}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 87s 4s/step - accuracy: 0.6223 - loss: 1.9830
Test accuracy: 0.7804
